This notebook is created for self-studying for Quantitative Macroeconomics course at NYU, taught by Gianluca Violante. Course website: https://sites.google.com/a/nyu.edu/glviolante/teaching/quantmacro

In [9]:
addpath('C:\Users\barry\Documents\Academics\Quantecon-Notes\functions')
cd 'C:\Users\barry\Documents\Academics\Quantecon-Notes'

# Lecture 2

## Differentiation
Finite difference: $f'(x_0) \simeq \frac{f(x_0+h)-f(x_0)}{h}$

Two-sided derivative: $f'(x_0) \simeq \frac{f(x_0+h)-f(x_0-h)}{2h}$. It is better as long as the fourth order residual on both sides of $x_0$ are of similar magnitude (prove using Taylor expansion)

In practice, compute TS as $f'(x_0) \simeq \frac{f(x_0+h)-f(x_0-h)}{x_0+h - (x_0-h)}$ so both have same argument. 

Miranda-Fackler: set $h = max(|x_0|,1)\sqrt[3]{\epsilon} \simeq \max(|x_0,1|)\times 6 \times 10^{-6}$, where $\epsilon$ is margion epsilon (for 64-bit machine, $\epsilon = 2^{-52}$).

**Example: Two-sided finite difference for single variable function**

In [1]:
%%file TS_diff.m
function TS = TS_diff(f,x0)
    
    h = max(abs(x0),1)*6.0*1e-6;
    
    TS = (f(x0+h)-f(x0-h))/(2*h);


Created file 'C:\Users\barry\Documents\Academics\Quantecon-Notes\TS_diff.m'.


In [2]:
TS_diff(@exp, 2)


ans =

    7.3891




**Example: Two-sided finite difference for Jacobian of an arbitrary function**

In [3]:
%%file functions/TS_jac.m
function fjac = TS_jac(f,x)
    
    h = max(abs(x),1)*6.0*1e-6;
    xh1 = x+h;
    xh0 = x-h;
    h = xh1-xh0;
    
    for j =1:length(x)
        xx = x; xx(j) = xh1(j); f1 = feval(f,xx);
        xx(j) = xh0(j); f0 = feval(f,xx);
        fjac(:,j) = (f1-f0)/h(j);
    end
        

Created file 'C:\Users\barry\Documents\Academics\Quantecon-Notes\functions\TS_jac.m'.


In [4]:
x = [1,2,3,5,1];
TS_jac(@sin, x)


ans =

    0.5403         0         0         0         0
         0   -0.4161         0         0         0
         0         0   -0.9900         0         0
         0         0         0    0.2837         0
         0         0         0         0    0.5403




## Root Finding

(See other notebook)

### The matching model of the labor market

Model setup: risk-neutral households that discount future at rate $\beta = 1/R$. Linear production function, which we can therefore think 1 firm only hires 1 worker (homogeneity). Aggregate productivity $z$ draws from $\Gamma(z',z)$ with $N$ values ($z$ is current state and $z'$ is any possible future state. $\Gamma$ is transition matrix). Employed workers receive a wage $w(z)$ and separate at rate $\delta$ (rate for leaving the firm). 

Firm's profit: $z-w(z)$

Unemployed workers receive benefit $b$ and search for vacancies.

In each period there are fraction $u$ of unemployed workers, and $v$ vacant jobs. Number of matches taking place per unit of time is given by aggregate CRS (constant return to scale) matching function: $m = m(u,v)$.

Unemployed worker moving to employment as a Poisson process with rate $p(\theta) = m/u$, and job vacancy filled with rate $q(\theta) = m/v$. $\theta = v/u$ is called labor market tightness. Flow cost for firm of posting a vacancy is $\kappa$

Value of unemployed and employed workers at state $z$: 

$$U(z) = b + \beta \mathbb{E}_z[p(\theta)W(z') + (1-p(\theta))U(z')]$$

$$W(z) = w(z) + \beta \mathbb{E}_z[\delta U(z') + (1-\delta)W(z')]$$

Value of vacant and matched firms at state $z$:

$$V(z) = -\kappa + \beta \mathbb{E}_z[q(\theta)J(v') + (1-q(\theta))V(z')]$$

$$J(z) = z - w(z) + \beta \mathbb{E}_z[\delta V(z') + (1-\delta) J(z')]$$

Free entry of the firm: $V(z) = 0 \forall z$

**Solving for $\theta$:**

Wage $w(z)$ is determined by Nash bargaining between worker and the firm. Wage solves $\eta$ is bargaining power for worker

$$\max_{w(z)} [W(z)-U(z)]^\eta[J(z)-V(z)]^{1-\eta}$$

Solution is $w(z) = \eta z + (1-\eta)b + \eta \kappa \theta(z)$ (use chain rule to take FOC). 

After some algebra (use V(z) = 0):

$$\frac{\kappa}{\beta q(\theta(z_i))} = \sum_{j=1}^N [(1-\eta)(z_i-b)-\eta\kappa\theta(z_j)+\frac{(1-\delta)\kappa}{q(\theta(z_j))}]\Gamma(z_j,z_i)$$

which is a system of $N$ nonlinear equations in $N$ unknowns $\theta(z_i)$. Given $u_t$ and solve for $\theta_t$, we can simulate $v_t = u_t \theta_t$, $w_t$, and recall the law of motion for the endogenous state $u_{t+1} = u_t + \delta(1-u_t) - p(\theta_t)u(t)$   

**Full solution in homework (Todo)**

**Example: Cournot duopoly**

The inverse demand function is $p=q^{-1/\eta}$ and the two firms faces cost function $C_i(q_i)=\frac{1}{2}c_iq_i^2$ for $i=1,2$. The profit of the firm is $\pi_i(q_1, q_2) = P(q_1+q_2)q_i - C(q_i)$ and it solves the FOC: $\frac{\partial \pi_i}{\partial q_i} = 0$

In [6]:
%%file functions/cournot.m

function fval = cournot(q)
    c = [0.6;0.8]; eta = 1.6;
    e = -1/eta;
    fval = sum(q)^e + e*sum(q)^(e-1)*q - diag(c)*q;

Created file 'C:\Users\barry\Documents\Academics\Quantecon-Notes\functions\cournot.m'.


In [8]:
fsolve(@cournot,[0.2;0.2])


Equation solved.

fsolve completed because the vector of function values is near zero
as measured by the value of the function tolerance, and
the problem appears regular as measured by the gradient.


ans =

    0.8396
    0.6888




## Integration

Overview: three methods
1. Newton-Cotes methods: employ piecewise polynomial approximations to the integrand with evenly spaced nodes
2. Gaussian Quadrature: choose modes and weights that satisfy some moment-matching conditions
3. Monte-Carlo methods: use equally weighted random nodes

### Newton-Cotes: 
Basic idea: approximate $f$ with a piecewise linear polynomial $\tilde{f}$ whose integral is easy to compute. 

Trapezoid rule: $f(x) \simeq f(x_i) + \frac{f(x_{i+1})-f(x_i)}{x_{i+1} - x_i}(x-x_i)$ and $\int_a^b f(x)dx = \sum_{i=0}^N w_i \frac{f(x_{i+1})+f(x_i)}{2}$ where $w_i = x_{i+1} - x_i$. At the end use $w_i = 1/2$

Simpson rule: based on quadratic approximation. For smooth function, Simpson rule is better than trapezoid rule. But for nondifferentiable function, use trapezoid.

### Gaussian quadrature

A function can be approximated by a polynomial, and weights and quadrature points can be chosen efficiently. 

Let $\Phi = {\phi_k(x)}^\infty_{k=0}$ be a family of polynomials defined on $[a,b]$, $\phi_k(x) = \sum_{i=0}^k \alpha_{k,i}x^i$.

Definition: $\Phi$ is orghogonal wrt the weight function $w$ if $<\phi_k,\phi_j> = \int_a^b \phi_k(x)\phi_j(x)w(x)dx = 0$ for all $k \neq j$. If $<\phi_k,\phi_k> = 1$, then it is called orthonormal. 

Theorem: Suppose $\Phi$ is orghonormal wrt $w(x)$ and ${x_i}_1^n$ be the zeros of $\phi_n(x)$. Then $$\int_a^b f(x)w(x)dx = \sum_{i=1}^n w_if(x_i) + \frac{f(\xi)^{2n}}{2n!}$$
for some $\xi \in [a,b]$ and the weights are given by $$w_i = -\frac{1}{\phi_n'(x_i)\phi_{n+1}(x_i)} > 0$$

The thoerem gives zeros of orthonormal polynomials as quadrature nodes weights. In practice, use well-known polynomial families. 

<img src="misc/polynomial quadrature.PNG" style="width: 500px;">

### Monte-Carlo

Useful for high dimensional integration. 

Basic idea: from LLN, $\lim_{n\rightarrow\infty}\frac{1}{n}\sum_{i=1}^n f(x_i) = \int_0^1 f(x)dx$. 

Method: draw $n$ realizations of $x$ and compute the sum $I_n = \frac{1}{n}\sum_{i=1}^n f(x_i)$

Variance-reduction techniques:
1. Stratified sampling: divide $[0,1]$ into two subintervals $[0,\alpha]$ and $[\alpha, 1]$ in order to avoid concentration of draws in one subinterval from bad luck and use $I_n = \frac{\alpha}{n} \sum_{i=1}^n f(x_{1i}) + \frac{1-\alpha}{n}\sum_{i=1}^n f(x_{2i})$
2. Antithetic variance: if f is weakly increasing then $f(x)$ and $f(1-x)$ are negatively correlated. Therefore, compute $I_n = \frac{1}{2n}\sum_{i=1}^n [f(x_i) + f(1-x_i)]$